In [1]:
import numpy as np
import data
from timeit import default_timer as timer

In [2]:
train = data.train
test = data.test

In [3]:
class MatrixFactorization():
    
    def __init__(self, train, test, k, learning_rate, reg_param, epochs, verbose = False):
        """
        param R : Rating Matrix
        param k : latent parameter
        param learning_rate : alpha on weight update
        param reg_param : regularization parameter
        param epochs : training epochs
        param verbose : print status
        """
        
        self._R = train
        self._test = test
        self._I = np.array(np.vectorize(lambda x: 0 if x==0 else 1)(train), dtype = np.float64)
        self._mask = self._I == 1 
        self._N = 1./np.linalg.norm(self._I, axis=1)
        self._num_users, self._num_items = test.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose
        
        
    def fit(self):
        """
        training Matrix Factorization : update matrix latent weight and bias
        """
        
        # init latent features _ how to initialize?
        self._P = np.random.normal(scale = 1.0/self._k, size=(self._num_users, self._k))
        self._Q = np.random.normal(scale = 1.0/self._k, size=(self._num_items, self._k))
        self._Y = np.random.normal(scale = 1.0/self._k, size=(self._num_items, self._k))\
        
        # init biases
        self._b_P = np.zeros(self._num_users)
        self._b_Q = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R != 0)]) # 0이 아닌 rating에 대해 평균
        
        # train while epochs
        self._training_process = []
        time = 0
        start_time = timer()
        for epoch in range(self._epochs):
            # rating이 존재하는 index를 기준으로 training
            start = timer()
            for u in range(self._num_users):
                for i in range(self._num_items):
                    if self._R[u, i] > 0:
                        self.gradient_descent(u, i, self._R[u, i])
            time += timer() - start
            train_cost, test_cost = self.cost()
            self._training_process.append((epoch, train_cost, test_cost, time))
            
            if self._verbose == True and ((epoch + 1) % 10 == 0 ):
                print("Iteration : %d, train_cost = %.4f, test_cost = %.4f, average time for 1 epoch : %.4f" % (epoch+1, train_cost, test_cost, time / 10))
                time = 0
        
        print("Total time for every epochs : %.4f" % (timer()-start_time))
                
        
    
    def cost(self):
        """
        compute RMSE
        """
        
        xi, yi = self._R.nonzero() # 0 이 아닌 값의 index 반환
        test_x, test_y = self._test.nonzero()
        
        cost_train = 0
        cost_test = 0

        for x, y in zip(xi, yi):
            cost_train += pow(self._R[x, y] - self.get_prediction(x, y), 2)
        
        for i, j in zip(test_x, test_y):
            cost_test += pow(self._test[i, j] - self.get_prediction(i, j), 2)

        return np.sqrt(cost_train/len(xi)), np.sqrt(cost_test/len(test_x))
    
        
    def gradient(self, error, u, i):
        """
        gradient of latent feature for GD
        param error : rating - prediction error
        param u : user index
        param i : item index
        """
        
        dp = (error * self._Q[i, :]) - (self._reg_param * self._P[u, :])
        dq = (error * (self._P[u, :] + self._N[u] * self._Y[self._mask[u,:], :].sum(axis=0))) - (self._reg_param * self._Q[i, :])
        
        temp = error * self._N[u] * self._Q[i, :]
        j = self._I[u, :].nonzero()
        dy = temp - self._reg_param * self._Y[j, :]
        
        return dp, dq, dy, j
    
    
    def gradient_descent(self, u, i, rating):
        """
        gradient descent function
        param u : user index
        param i : item index
        param rating : rating of (u, i)
        """        
        
        prediction = self.get_prediction(u, i)
        error = rating - prediction
        
        self._b_P[u] += self._learning_rate * (error - self._reg_param * self._b_P[u])
        self._b_Q[i] += self._learning_rate * (error - self._reg_param * self._b_Q[i])
        
        dp, dq, dy, j = self.gradient(error, u, i)
        self._P[u, :] += self._learning_rate * dp
        self._Q[i, :] += self._learning_rate * dq
        self._Y[j, :] += self._learning_rate * dy
        
        
    
    def get_prediction(self, u, i):
        """
        get predicted rating by user i on item j
        """
        
        x = self._b + self._b_P[u] + self._b_Q[i] + self._Q[i, :].T.dot(self._P[u, :] + self._N[u] * self._Y[self._mask[u,:], :].sum(axis=0))
        
        return x

    
    def get_complete_matrix(self):
        """
        compute complete matrix
        """
        
        predictions = np.zeros([self._num_users, self._num_items])
        
        # cost 구할 때 bottle neck이 되는 지점 
        for u in range(self._num_users):
            for i in range(self._num_items):
                predictions[u, i] = self.get_prediction(u, i)
        
        return predictions
    
    
    def print_results(self):
        """
        print fit results
        """

        print("Final R matrix:")
        print(self.get_complete_matrix())
        print("Final RMSE:")
        print(self._training_process[self._epochs-1][2])

In [ ]:
np.random.seed(7)
np.seterr(all="warn")
    
factorizer = MatrixFactorization(train, test, k=40, learning_rate=0.001, reg_param=0.001, epochs=100, verbose=True)

# regression parameter 2개
factorizer.fit()
factorizer.print_results()